# Homework 4

## Imports

In [1]:
import nbtools
nbtools.setup_nb()

In [2]:
from astropy import units
import itertools

import numpy
import pandas
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

import sympy
from sympy.diffgeom import Manifold, Patch, TensorProduct as tprod

from pystein import coords, metric, curvature
from pystein.utilities import tensor_pow as tpow, full_simplify

## Theory

### A1

Part $(ii)$

In [54]:
M = Manifold('M', dim=4)
P = Patch('origin', M)

t, r, theta, phi = sympy.symbols('t r theta phi')
A = sympy.Function('A')(t, r)
B = sympy.Function('B')(t, r)

cs = coords.CoordSystem('Schwarzschild', P, [t, r, theta, phi])
dt, dr, dtheta, dphi = cs.base_oneforms()

ds2 = (- sympy.exp(2 * A) * tpow(dt, 2) 
       + sympy.exp(2 * B) * tpow(dr, 2) 
       + r ** 2 * (tpow(dtheta, 2) + sympy.sin(theta) ** 2 * tpow(dphi, 2)))

g = metric.Metric(twoform=ds2)
g

r**2*(sin(theta)**2*TensorProduct(dphi, dphi) + TensorProduct(dtheta, dtheta)) - exp(2*A(t, r))*TensorProduct(dt, dt) + exp(2*B(t, r))*TensorProduct(dr, dr)

In [56]:
G_mu_nu = [(mu, nu, curvature.einstein_tensor_component(mu, nu, g)) 
           for mu, nu in itertools.product(range(2), range(2))]

In [59]:
full_simplify(G_mu_nu[0][2].doit())

(2*r*Derivative(B(t, r), r) + exp(2*B(t, r)) - 1)*exp(2*A(t, r) - 2*B(t, r))/r**2

In [60]:
full_simplify(G_mu_nu[1][2].doit())

2*Derivative(B(t, r), t)/r

In [61]:
full_simplify(G_mu_nu[3][2].doit())

(2*r*Derivative(A(t, r), r) - exp(2*B(t, r)) + 1)/r**2

In [62]:
full_simplify(curvature.christoffel_symbol_component(1, 0, 0, g))

exp(2*A(t, r) - 2*B(t, r))*Derivative(A(t, r), r)

### A2

In [70]:
M = Manifold('M', dim=4)
P = Patch('origin', M)

v, r, theta, phi = sympy.symbols('v r theta phi')
alpha = sympy.Function('alpha')(v, r)
beta = sympy.Function('beta')(v, r)

cs = coords.CoordSystem('Eddington-Finkelstein', P, [v, r, theta, phi])
dv, dr, dtheta, dphi = cs.base_oneforms()

ds2 = (- alpha * tpow(dv, 2) 
       + beta * tprod(dv, dr) 
       + beta * tprod(dr, dv) 
       + r ** 2 * (tpow(dtheta, 2) + sympy.sin(theta) ** 2 * tpow(dphi, 2)))

g2 = metric.Metric(twoform=ds2)
g2

r**2*(sin(theta)**2*TensorProduct(dphi, dphi) + TensorProduct(dtheta, dtheta)) - alpha(v, r)*TensorProduct(dv, dv) + beta(v, r)*TensorProduct(dv, dr) + beta(v, r)*TensorProduct(dr, dv)

In [71]:
G_mu_nu = [(mu, nu, curvature.einstein_tensor_component(mu, nu, g2)) 
           for mu, nu in itertools.product(range(2), range(2))]

In [72]:
full_simplify(G_mu_nu[0][2].doit())

(r*(-Derivative(alpha(v, r), r) + 2*Derivative(beta(v, r), v))*alpha(v, r)*beta(v, r) + 2*r*alpha(v, r)**2*Derivative(beta(v, r), r) - r*beta(v, r)**2*Derivative(alpha(v, r), v) - alpha(v, r)**2*beta(v, r) + alpha(v, r)*beta(v, r)**3)/(r**2*beta(v, r)**3)

In [73]:
full_simplify(G_mu_nu[1][2].doit())

(-2*r*alpha(v, r)*Derivative(beta(v, r), r) + r*beta(v, r)*Derivative(alpha(v, r), r) + alpha(v, r)*beta(v, r) - beta(v, r)**3)/(r**2*beta(v, r)**2)

In [75]:
full_simplify(G_mu_nu[3][2].doit())

2*Derivative(beta(v, r), r)/(r*beta(v, r))

The above the says that $\beta = \beta(v)$, which can simplify $G_{vv}$ and $G_{vr}$

In [77]:
beta_simp = sympy.Function('beta')(v)
ds2_simp = (- alpha * tpow(dv, 2) 
       + beta_simp * tprod(dv, dr) 
       + beta_simp * tprod(dr, dv) 
       + r ** 2 * (tpow(dtheta, 2) + sympy.sin(theta) ** 2 * tpow(dphi, 2)))

g2_simp = metric.Metric(twoform=ds2_simp)

In [78]:
G_mu_nu_simp = [(mu, nu, curvature.einstein_tensor_component(mu, nu, g2_simp)) 
                 for mu, nu in itertools.product(range(2), range(2))]

In [79]:
full_simplify(G_mu_nu_simp[0][2].doit())

(r*(-Derivative(alpha(v, r), r) + 2*Derivative(beta(v), v))*alpha(v, r) - r*beta(v)*Derivative(alpha(v, r), v) - alpha(v, r)**2 + alpha(v, r)*beta(v)**2)/(r**2*beta(v)**2)

In [80]:
full_simplify(G_mu_nu_simp[1][2].doit())

(r*Derivative(alpha(v, r), r) + alpha(v, r) - beta(v)**2)/(r**2*beta(v))

The above gives equation for $\alpha$ as $G_{vr} = 0$
$$ \frac{1}{r^2 \beta} \left[ r\partial_r \alpha + \alpha - \beta^2 \right] = 0 \implies  r\partial_r \alpha + \alpha - \beta^2 = 0 \implies \alpha = \left[1 + \frac{f(v)}{r}\right] \beta^2(v)$$

In [81]:
f = sympy.Function('f')(v)

# Second simplification to metric
alpha_simp = (1 + f / r) * beta_simp ** 2
ds2_simp2 = (- alpha_simp * tpow(dv, 2) 
             + beta_simp * tprod(dv, dr) 
             + beta_simp * tprod(dr, dv) 
             + r ** 2 * (tpow(dtheta, 2) + sympy.sin(theta) ** 2 * tpow(dphi, 2)))

g2_simp2 = metric.Metric(twoform=ds2_simp2)

In [82]:
G_mu_nu_simp2 = [(mu, nu, curvature.einstein_tensor_component(mu, nu, g2_simp2)) 
                  for mu, nu in itertools.product(range(2), range(2))]

In [84]:
full_simplify(G_mu_nu_simp2[0][2].doit())

-beta(v)*Derivative(f(v), v)/r**2

In [85]:
Phi = sympy.Function('Phi')(r)
ds2_temp = (- (1 + 2 * Phi) * tpow(dv, 2) 
             + tprod(dv, dr) 
             + tprod(dr, dv) 
             + r ** 2 * (tpow(dtheta, 2) + sympy.sin(theta) ** 2 * tpow(dphi, 2)))

g2_temp = metric.Metric(twoform=ds2_temp)
g2_temp

r**2*(sin(theta)**2*TensorProduct(dphi, dphi) + TensorProduct(dtheta, dtheta)) + (-2*Phi(r) - 1)*TensorProduct(dv, dv) + TensorProduct(dv, dr) + TensorProduct(dr, dv)

In [86]:
g2_temp.matrix

Matrix([
[-2*Phi(r) - 1, 1,    0,                  0],
[            1, 0,    0,                  0],
[            0, 0, r**2,                  0],
[            0, 0,    0, r**2*sin(theta)**2]])

In [87]:
g2_temp.inverse.matrix

Matrix([
[0,            1,       0,                      0],
[1, 2*Phi(r) + 1,       0,                      0],
[0,            0, r**(-2),                      0],
[0,            0,       0, 1/(r**2*sin(theta)**2)]])

In [101]:
tmp_00 = curvature.einstein_tensor_component(0, 0, g2_temp)

In [102]:
full_simplify(tmp_00.doit())

-2*(r*Derivative(Phi(r), r) + Phi(r))*(2*Phi(r) + 1)/r**2

### A3

In [3]:
M = Manifold('M', dim=4)
P = Patch('origin', M)

t, r, theta, phi = sympy.symbols('t r theta phi')
A = sympy.Function('A')(r)
B = sympy.Function('B')(r)

cs = coords.CoordSystem('Schwarzschild', P, [t, r, theta, phi])
dt, dr, dtheta, dphi = cs.base_oneforms()

ds2 = (- sympy.exp(2 * A) * tpow(dt, 2) 
       + sympy.exp(2 * B) * tpow(dr, 2) 
       + r ** 2 * (tpow(dtheta, 2) + sympy.sin(theta) ** 2 * tpow(dphi, 2)))

g = metric.Metric(twoform=ds2)
g

r**2*(sin(theta)**2*TensorProduct(dphi, dphi) + TensorProduct(dtheta, dtheta)) - exp(2*A(r))*TensorProduct(dt, dt) + exp(2*B(r))*TensorProduct(dr, dr)

In [4]:
crs = curvature.compute_components(g, riemann=False, ricci=False)

In [5]:
curvature.display_components(crs)

In [8]:
full_simplify(sympy.exp(-2 * A) * curvature.einstein_tensor_component(0, 0, g).doit())

(2*r*Derivative(B(r), r) + exp(2*B(r)) - 1)*exp(-2*B(r))/r**2

In [9]:
full_simplify(sympy.exp(-2 * B) * curvature.einstein_tensor_component(1, 1, g).doit())

(2*r*Derivative(A(r), r) - exp(2*B(r)) + 1)*exp(-2*B(r))/r**2

In [10]:
full_simplify(sympy.exp(-2 * A) * curvature.einstein_tensor_component(0, 0, g).doit() +
              sympy.exp(-2 * B) * curvature.einstein_tensor_component(1, 1, g).doit())

2*(Derivative(A(r), r) + Derivative(B(r), r))*exp(-2*B(r))/r

## Exercises

### B2

In [91]:
M = Manifold('M', dim=4)
P = Patch('origin', M)

t, r, theta, phi, r_s = sympy.symbols('t r theta phi r_s')

cs = coords.CoordSystem('Schwarzschild', P, [t, r, theta, phi])
dt, dr, dtheta, dphi = cs.base_oneforms()

schwarz = (1 - r_s / r)

ds2 = (- schwarz * tpow(dt, 2) 
       + (schwarz ** -1)  * tpow(dr, 2) 
       + r ** 2 * (tpow(dtheta, 2) + sympy.sin(theta) ** 2 * tpow(dphi, 2)))

g = metric.Metric(twoform=ds2)
g

r**2*(sin(theta)**2*TensorProduct(dphi, dphi) + TensorProduct(dtheta, dtheta)) + (-1 + r_s/r)*TensorProduct(dt, dt) + TensorProduct(dr, dr)/(1 - r_s/r)

In [93]:
full_simplify(curvature.ricci_scalar(g, simplify_intermediate=False).doit())

0

In [4]:
crs = curvature.compute_components(g, riemann=False, ricci=False)

In [5]:
1e8 / (365.25 * 24 * 60 * 60)

3.168808781402895

In [6]:
curvature.display_components(crs)

In [8]:
def a_norm(r, r_s):
    return 0.5 * (r_s / (r ** 2) ) * (1 / numpy.sqrt(1 - (r_s / r)) )

In [26]:
rs = numpy.arange(1 + 0.0001, 2 + 0.001, 0.0001)
df = pandas.DataFrame({'radius': rs, 'a_norm': a_norm(rs, 1)})
fig = px.line(df, x='radius', y='a_norm')
fig.update_layout(yaxis_range=[0,5],
                  xaxis_range=[0.8,2],
                  width=700,
                  height=500,
                  showlegend=False,
                  title_text=r'$\text{Stationary Observer Acceleration in Schwarzschild }(t, r)$', 
                  title_x=0.5,
                  xaxis_title=r'$r / r_s$',
                  yaxis_title=r'$r_s a \equiv r_s \sqrt{g_{\mu\nu}a^\mu a^\nu}$')
fig.show()

In [28]:
ep = sympy.symbols('varepsilon')
a_ep = sympy.Rational(1, 2) * r_s / ( (r_s + ep) ** 2 ) * sympy.sqrt((r_s + ep) / ep)
a_ep

r_s*sqrt((r_s + varepsilon)/varepsilon)/(2*(r_s + varepsilon)**2)

In [39]:
sympy.series(a_ep, x=ep, n=0)

1/(2*sqrt(r_s)*sqrt(varepsilon)) + O(1)

### B3

In [54]:
r_s, r = sympy.symbols('r_s r', real=True, nonnegative=True)
dt_dr = 1 / (1 - r_s / r)
dt_dr

1/(1 - r_s/r)

Ingoing

In [55]:
sympy.integrate(dt_dr, r)

r + r_s*log(r - r_s)

In [56]:
sympy.integrate(-dt_dr, r)

-r - r_s*log(r - r_s)

In [57]:
def ring(r, r_s, c):
    return -r - r_s * numpy.log(numpy.abs(r - r_s)) + c

def rong(r, r_s, c):
    return r + r_s * numpy.log(numpy.abs(r - r_s)) + c

In [60]:
_r_s = 1.0
_ep = 1e-3
_r = numpy.arange(0*_r_s + _ep, 5 * _r_s + _ep, _ep)

_c = numpy.arange(-1.0, 1.0, 1.0)

dfs = []
for c in _c:
    df_i = pandas.DataFrame({'r': _r, 't': ring(_r, _r_s, c)}).assign(c=c, kind='In')
    df_o = pandas.DataFrame({'r': _r, 't': rong(_r, _r_s, c)}).assign(c=c, kind='Out')
    dfs.extend([df_i, df_o])
df = pandas.concat(dfs, axis=0)

/var/folders/4_/_vr7p6557_3_5cyncn6tdq_w0000gn/T/ipykernel_50326/3021935099.py:2: RuntimeWarning:

divide by zero encountered in log

/var/folders/4_/_vr7p6557_3_5cyncn6tdq_w0000gn/T/ipykernel_50326/3021935099.py:5: RuntimeWarning:

divide by zero encountered in log



In [62]:
fig = px.line(df, x='r', y='t', color='c', line_dash='kind')
fig.update_layout(yaxis_range=[-5,5],
                  xaxis_range=[0,5],
                  width=800,
                  height=600,
                  showlegend=False,
                  title_text=r'$\text{Null Radial Geodesics in Schwarzschild }(t, r)$', 
                  title_x=0.5,
                  xaxis_title=r'$r / r_s$',
                  yaxis_title=r'$t(r)$')
fig.show()

### B4

In [13]:
def z(r_e, r_r, r_s = 1.0):
    return numpy.sqrt(1 - (r_s / r_r)) / numpy.sqrt(1 - (r_s / r_e)) - 1

In [36]:
z(1.01, 10)

8.534149149242417

In [51]:
r_e , r_r = numpy.meshgrid(numpy.arange(1.0, 1.01, 0.0001),
                           numpy.arange(1.0, 2.0, 0.01))
zs = z(r_e, r_r)

/var/folders/4_/_vr7p6557_3_5cyncn6tdq_w0000gn/T/ipykernel_50326/3858421975.py:2: RuntimeWarning:

divide by zero encountered in true_divide

/var/folders/4_/_vr7p6557_3_5cyncn6tdq_w0000gn/T/ipykernel_50326/3858421975.py:2: RuntimeWarning:

invalid value encountered in true_divide



In [52]:
df = pandas.DataFrame({'r_e': r_e.ravel(), 'r_r': r_r.ravel(), 'z':zs.ravel()})
df = df[df['r_r'] >= df['r_e']]
fig = px.scatter(df, x='r_e', y='r_r', color='z')
fig_ep = 0.25
fig.update_layout(
#     yaxis_range=[1.0 - fig_ep, 5.0 + fig_ep],
#                   xaxis_range=[1.0 - fig_ep, 5.0 + fig_ep],
                  width=600,
                  height=600,
#                   showlegend=False,
                  title_text=r'$\text{Gravitational Redshift in Schwarzschild }(t, r)$', 
                  title_x=0.5,
                  xaxis_title=r'$r_E$',
                  yaxis_title=r'$r_R$')
fig.show()

## Problem